In [8]:
import codecs
import operator
from typing import List

def bytes2hex(b: bytes) -> bytes:
    #return b.hex().encode()
    return codecs.encode(b, 'hex')
    
def str2hex(s: str, encoding='utf-8') -> bytes:
    return bytes2hex(s.encode(encoding))

def hex2bytes(x: bytes) -> bytes:
    #return bytes.fromhex(x.decode())
    return codecs.decode(x, 'hex')


def int2hex(x: int, encoding='utf-8') -> bytes:
    return '{:02x}'.format(x).encode(encoding)

def hex2int(x: str) -> int:
    return int(x, 16)

assert str2hex("the kids don't play") == b'746865206b69647320646f6e277420706c6179'
assert hex2bytes(str2hex("the kids don't play")) == b"the kids don't play"
assert int2hex(255) == b'ff'
assert hex2int(int2hex(255)) == 255

# standard byte transformations
assert bytes.fromhex("the kids don't play".encode().hex()) == b"the kids don't play"

# just some notes on int conversion
assert int.from_bytes(b'1c0111001f010100061a024b53535009181c', byteorder='big').to_bytes(36, byteorder='big') == b'1c0111001f010100061a024b53535009181c'

#type(list(zip(b'1c0111001f010100061a024b53535009181c', b'686974207468652062756c6c277320657965'))[0][0])

In [9]:
# s1c1
# http://cryptopals.com/sets/1/challenges/3
import base64
import codecs

def hex_to_b64(b: bytes) -> str:
    return base64.b64encode(hex2bytes(b))

hex_to_b64(b'49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d') == b'SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t'

True

In [10]:
# s1c2
# http://cryptopals.com/sets/1/challenges/2
# Write a function that takes two equal-length buffers and produces their XOR combination.
import itertools
from typing import Tuple

b1 = b'1c0111001f010100061a024b53535009181c'
b2 = b'686974207468652062756c6c277320657965'

def xor_bytes(b1: bytes, b2: bytes, padding: int=0) -> bytes:
    return bytes(i^j for i,j in itertools.zip_longest(b1, b2, fillvalue=padding))

def xor_hex(b1: bytes, b2: bytes, padding: int=0) -> str:
    return bytes2hex(bytes(i^j for i,j in itertools.zip_longest(hex2bytes(b1), hex2bytes(b2), fillvalue=padding)))

print(bytes2hex(xor_bytes(hex2bytes(b1), hex2bytes(b2))))

# test
bytes2hex(xor_bytes(hex2bytes(b1), hex2bytes(b2))) == b'746865206b696420646f6e277420706c6179'


b'746865206b696420646f6e277420706c6179'


True

In [11]:
# s1c3
# http://cryptopals.com/sets/1/challenges/3
# Decrypt the string that has been xor'd with a single character
import string
from typing import Callable

b = b'1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736'


frequency = {
    'a': 0.0651738,
    'b': 0.0124248,
    'c': 0.0217339,
    'd': 0.0349835,
    'e': 0.1041442,
    'f': 0.0197881,
    'g': 0.0158610,
    'h': 0.0492888,
    'i': 0.0558094,
    'j': 0.0009033,
    'k': 0.0050529,
    'l': 0.0331490,
    'm': 0.0202124,
    'n': 0.0564513,
    'o': 0.0596302,
    'p': 0.0137645,
    'q': 0.0008606,
    'r': 0.0497563,
    's': 0.0515760,
    't': 0.0729357,
    'u': 0.0225134,
    'v': 0.0082903,
    'w': 0.0171272,
    'x': 0.0013692,
    'y': 0.0145984,
    'z': 0.0007836,
    ' ': 0.1918182,}


def score_engrams(plain: bytes) -> float:
    return sum(frequency.get(i, 0) for i in plain.decode(errors='ignore'))

def decode_single_xor(cipher: bytes, score_func: Callable=score_engrams):
    for i in range(255):
        plain = xor_bytes(cipher, [i], padding=int(i))
        yield score_engrams(plain), plain, i


print(max(decode_single_xor(hex2bytes(b))))
assert max(decode_single_xor(hex2bytes(b)))[1] == b"Cooking MC's like a pound of bacon"

(2.2004246999999997, b"Cooking MC's like a pound of bacon", 88)


In [12]:
# s1c4
# http://cryptopals.com/sets/1/challenges/4
# Detect single-character XOR from file
from typing import List

bigram_freq = {
 'th': 1.52,
 'en': 0.55,
 'ng': 0.18,
 'he': 1.28,
 'ed': 0.53,
 'of': 0.16,
 'in': 0.94,
 'to': 0.52,
 'al': 0.09,
 'er': 0.94,
 'it': 0.50,
 'de': 0.09,
 'an': 0.82,
 'ou': 0.50,
 'se': 0.08,
 're': 0.68,
 'ea': 0.47,
 'le': 0.08,
 'nd': 0.63,
 'hi': 0.46,
 'sa': 0.06,
 'at': 0.59,
 'is': 0.46,
 'si': 0.05,
 'on': 0.57,
 'or': 0.43,
 'ar': 0.04,
 'nt': 0.56,
 'ti': 0.34,
 've': 0.04,
 'ha': 0.56,
 'as': 0.33,
 'ra': 0.04,
 'es': 0.56,
 'te': 0.27,
 'ld': 0.02,
 'st': 0.55,
 'et': 0.19,
 'ur': 0.02,
}

def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = itertools.tee(iterable)
    next(b, None)
    return zip(a, b)

def score_bigrams(plain: str) -> float:
    score = sum(frequency.get(i, 0) for i in plain)
    score += sum(bigram_freq.get(i+j, 0) for i,j in pairwise(plain))
    return score

f = open('4.txt')
ciphers = list(map(str.strip, f.readlines()))


def decypher_list_of_xors(ciphers: List[str]):
    for cipher in ciphers:
        yield from decode_single_xor(hex2bytes(cipher.encode()), score_bigrams)
            
result = max(decypher_list_of_xors(ciphers))
print(result)
assert result[1] == b'Now that the party is jumping\n'

(2.0316805000000002, b'Now that the party is jumping\n', 53)


In [13]:
# s1c5
# http://cryptopals.com/sets/1/challenges/5
# Encrypt with a repeating key
plain = b'''Burning 'em, if you ain't quick and nimble
I go crazy when I hear a cymbal'''

import itertools

def encrypt_repeating(plain: bytes, key: bytes):
        return xor_bytes(plain, bytes(itertools.islice(itertools.cycle(key), len(plain))))
      
print(bytes2hex(encrypt_repeating(plain, b'ICE')))
# test
assert bytes2hex(encrypt_repeating(plain, b'ICE')) == b'0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f'

b'0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f'


In [14]:
# s1c6
# http://cryptopals.com/sets/1/challenges/6
# Decrypt 6.txt (it's been encrypted with a repeating XOR and base64'd)

def hamming_distance(b1: bytes, b2: bytes) -> int:
    return sum(bin(i^j).count('1') for i, j in zip(b1, b2))

assert hamming_distance(b'this is a test', b'wokka wokka!!!') == 37, 'Hamming distance fails'

def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]


def most_likely_key(cipher, min_key=2, max_key=40):
    max_key = int(min(max_key, len(cipher)/4)) # Don't accept keys that are too large.
    best = (2**63-1, None)  # Infinite score
    for k in range(min_key, max_key+1):
        blocks = itertools.islice(chunks(cipher, k), 4)  # Just use the first 4 blocks of k length
        pairs = itertools.combinations(blocks, r=2)
        distances = list(hamming_distance(i,j)/k for i,j in pairs)
        if not distances:
            break
        score = sum(distances)/len(distances)
        best = min(best, (score, k))
    return best
    

def break_repeating(cipher):
    keysize = most_likely_key(cipher)[1]
    t_blocks = itertools.zip_longest(*chunks(cipher, keysize), fillvalue=0)  # Transpose the blocks
    keychars = []
    for block in (bytes(i) for i in t_blocks): 
        keychars.append(max(decode_single_xor(block))[2])
    return bytes(keychars)


# Test
cipher = encrypt_repeating(b'this is a test and a longer test', b'ABC')
print('keysize: {}'.format(most_likely_key(cipher)))
print('key: {}'.format(break_repeating(cipher)))
print('decrypted: {}'.format(encrypt_repeating(cipher, break_repeating(cipher))))
print()

cipher = base64.b64decode(open('6.txt').read())
print('keysize: {}'.format(most_likely_key(cipher)))
print('key: {}'.format(break_repeating(cipher)))
print('decrypted: {}'.format(encrypt_repeating(cipher, break_repeating(cipher))))

keysize: (2.055555555555556, 3)
key: b'ABC'
decrypted: b'this is a test and a longer test'

keysize: (2.7471264367816093, 29)
key: b'Terminator X: Bring the noise'
decrypted: b"I'm back and I'm ringin' the bell \nA rockin' on the mike while the fly girls yell \nIn ecstasy in the back of me \nWell that's my DJ Deshay cuttin' all them Z's \nHittin' hard and the girlies goin' crazy \nVanilla's on the mike, man I'm not lazy. \n\nI'm lettin' my drug kick in \nIt controls my mouth and I begin \nTo just let it flow, let my concepts go \nMy posse's to the side yellin', Go Vanilla Go! \n\nSmooth 'cause that's the way I will be \nAnd if you don't give a damn, then \nWhy you starin' at me \nSo get off 'cause I control the stage \nThere's no dissin' allowed \nI'm in my own phase \nThe girlies sa y they love me and that is ok \nAnd I can dance better than any kid n' play \n\nStage 2 -- Yea the one ya' wanna listen to \nIt's off my head so let the beat play through \nSo I can funk it up and make it 

In [16]:
# s1c7
# http://cryptopals.com/sets/1/challenges/7
# Decrypt 7.txt with the key "YELLOW SUBMARINE". It's been encrypted using AES-128
!pip install pycrypto
import crypto
import sys
sys.modules['Crypto'] = crypto
from Crypto.Cipher import AES

cipher = base64.b64decode(open('7.txt').read())
aes = AES.new('YELLOW SUBMARINE', AES.MODE_ECB)
aes.decrypt(cipher)

b"I'm back and I'm ringin' the bell \nA rockin' on the mike while the fly girls yell \nIn ecstasy in the back of me \nWell that's my DJ Deshay cuttin' all them Z's \nHittin' hard and the girlies goin' crazy \nVanilla's on the mike, man I'm not lazy. \n\nI'm lettin' my drug kick in \nIt controls my mouth and I begin \nTo just let it flow, let my concepts go \nMy posse's to the side yellin', Go Vanilla Go! \n\nSmooth 'cause that's the way I will be \nAnd if you don't give a damn, then \nWhy you starin' at me \nSo get off 'cause I control the stage \nThere's no dissin' allowed \nI'm in my own phase \nThe girlies sa y they love me and that is ok \nAnd I can dance better than any kid n' play \n\nStage 2 -- Yea the one ya' wanna listen to \nIt's off my head so let the beat play through \nSo I can funk it up and make it sound good \n1-2-3 Yo -- Knock on some wood \nFor good luck, I like my rhymes atrocious \nSupercalafragilisticexpialidocious \nI'm an effect and that you can bet \nI can take 

In [ ]:
# s1c7
# http://cryptopals.com/sets/1/challenges/7
# Detect which line on 8.txt has been encrypted with AES in ECB mode

